In [1]:
import gradio as gr
import requests
import time
from PIL import Image, ImageDraw, ImageFont
import random

FONT = ImageFont.truetype(font='malgun.ttf', size=18)

In [ ]:
API_KEY = '***'

def request_document_intelligence(file_path):
    endpoint = "https://fimtrus-document-intelligence.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read:analyze?_overload=analyzeDocument&api-version=2024-11-30"
    
    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY,
        "Content-Type": "image/png"
    }
    
    with open(file_path, "rb") as image:
        image_data = image.read()
    
    response = requests.post(endpoint, headers=headers, data=image_data)
    
    if response.status_code == 202:
        result_url = response.headers['Operation-Location']
        
        while(True):
            result_response = requests.get(result_url, headers={
                "Ocp-Apim-Subscription-Key": headers["Ocp-Apim-Subscription-Key"]
            })
            result_response_json = result_response.json()
            result_status = result_response_json['status']
            
            if result_status == "running":
                print(result_status)
                time.sleep(1)
                continue
            else:
                break
            
        if result_status == "succeeded":
            print(result_status)
            print(result_response_json)
            return result_response_json
        else:
            return None

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def draw_image(file_path, result_response):
    image = Image.open(file_path)
    draw = ImageDraw.Draw(image)
    
    block_list = result_response['analyzeResult']['paragraphs']
    
    for block in block_list:
        line_color = random_color()
        polygon_list = block['boundingRegions'][0]['polygon']
        points = [(polygon_list[i], polygon_list[i + 1]) for i in range(0, len(polygon_list), 2)]
        content = block['content']
        # point_list = list()
        # for i in range(0, len(polygon_list), 2):
        #     point_list.append((point_list[i], point_list[i + 1]))
        draw.polygon(points, outline=line_color, width=2)
        draw.text((points[0][0], points[0][1] - 10), content, fill=line_color, font=FONT)
        
    return image


with gr.Blocks() as demo:
    gr.Markdown('# 📸 문서에서 텍스트 검출')
    send_button = gr.Button('검출하기')
    with gr.Row():
        input_image = gr.Image(label="입력 이미지", type="filepath")
        output_image = gr.Image(label="출력 이미지", interactive=False, type="pil")
    
    def click_send(file_path):
        result_response = request_document_intelligence(file_path)
        image = draw_image(file_path, result_response)
        return image
    
    send_button.click(click_send, inputs=[input_image], outputs=[output_image])
    
demo.launch()
# file = "/Users/fimtrus/Documents/002.강의/005.AI Services/104.Document Intelligence 샘플 파일/read-healthcare.png"
# result_response = request_document_intelligence(file)
# draw_image(file, result_response)


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


running
succeeded
{'status': 'succeeded', 'createdDateTime': '2025-03-24T05:58:01Z', 'lastUpdatedDateTime': '2025-03-24T05:58:02Z', 'analyzeResult': {'apiVersion': '2024-11-30', 'modelId': 'prebuilt-read', 'stringIndexType': 'textElements', 'content': "v 5. CLASS\n[편집]\nsolved.ac에서 수준별로 엄선한 문제로, 2022년 8월 현재 CLASS 10까지 존재한다. 대부분 자료구조/알고리즘 기본 문제들과 비슷한 수준에서 도전해 보면 좋을 만한 문제들로 구성되어 있다. 단순히 문제를 많이 풀면 올라가는 티어와 달리 CLASS 문제들은 엄선된 특정 문제를 풀어야 하는 방식이기 때문에, 현재 자신의 실질적인 위치가 궁금하거나 빠른 실력향상을 원한다면 티어에 신경쓰기보다 이쪽을 먼저 공략해보는 것이 좋다.\nCLASS 1은 초보자들이 따라하기 쉬운 문제들로 구성되어 있으나, CLASS 3만 가도 머리를 싸매야 하는 문제들이 나온다. 주어진 개수만큼 문제를 풀면 해당 CLASS를 획득할 수 있으며, 에센셜 태그가 붙은 문제를 다 풀면 은장이, 모든 문제를 풀면 금장이 추가로 붙는다.\nCLASS 9부터는 이전 CLASS를 통과해야 취득할 수 있다.\n일반적인 실무를 위해서는 CLASS 3까지만 획득해도 충분하며, CLASS 4까지 취득하는 것은 구현력에 도움이 될 수 있다. [22] 프로그래밍 대회에 등장하는 기초 알고리 즘들을 배우고 싶다면 CLASS 5부터 차근차근 올라가보는 것도 도움이 된다.\n아래 표는 각 CLASS에서 새롭게 다루는 내용들을 정리한 것이다. 이 내용들에 추가로 이전 CLASS까지들에서 다뤘던 내용이 심화된 문제들로 구성되어 있다.\nCLASS\n새롭게 다루는 내용\nBOJ 단계별로 풀어보기\n1\n입출력, 사칙연산, 조건문, 반복문